# Proyecto 03 - Procesamiento del Lenguaje Natural

## Dataset: The Multilingual Amazon Reviews Corpus

**Recuerda descargar el dataset de [aquí](https://github.com/kang205/SASRec). Es un archivo .zip que contiene tres documentos. Más información sobre el dataset [aquí](https://registry.opendata.aws/amazon-reviews-ml/). Es importante que tengas en cuenta la [licencia](https://docs.opendata.aws/amazon-reviews-ml/license.txt) de este dataset.**

### Exploración de datos y Procesamiento del Lenguaje Natural

Dedícale un buen tiempo a hacer un Análisis Exploratorio de Datos. Considera que hasta que no hayas aplicado las herramientas de Procesamiento del Lenguaje Natural vistas, será difícil completar este análisis. Elige preguntas que creas que puedas responder con este dataset. Por ejemplo, ¿qué palabras están asociadas a calificaciones positivas y qué palabras a calificaciones negativas?

### Machine Learning

Implementa un modelo que, dada la crítica de un producto, asigne la cantidad de estrellas correspondiente. **Para pensar**: ¿es un problema de Clasificación o de Regresión?

1. Haz todas las transformaciones de datos que consideres necesarias. Justifica.
1. Evalúa de forma apropiada sus resultados. Justifica la métrica elegida.
1. Elige un modelo benchmark y compara tus resultados con este modelo.
1. Optimiza los hiperparámetros de tu modelo.
1. Intenta responder la pregunta: ¿Qué información está usando el modelo para predecir?

**Recomendación:** si no te resulta conveniente trabajar en español con NLTK, te recomendamos que explores la librería [spaCy](https://spacy.io/).

### Para pensar, investigar y, opcionalmente, implementar
1. ¿Valdrá la pena convertir el problema de Machine Learning en un problema binario? Es decir, asignar únicamente las etiquetas Positiva y Negativa a cada crítica y hacer un modelo que, en lugar de predecir las estrellas, prediga esa etiqueta. Pensar en qué situación puede ser útil. ¿Esperas que el desempeño sea mejor o peor?
1. ¿Hay algo que te gustaría investigar o probar?

### **¡Tómate tiempo para investigar y leer mucho!**

In [1]:
#Importo las librerias con las que voy a trabajar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.lang.es.examples import sentences
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import string
from sklearn.metrics import classification_report

**Teniendo en cuenta que los Datasets son de gran tamaño y tardan mucho en cargar, voy a usar el Dataset DEV para la demostracion de todas las transformaciones que hay que realizar y luego voy a importar los DS TRAIN Y TEST en formato CSV con las mismas ya realizadas**

## Dataset DEV ##

In [2]:
#dataset_dev = pd.read_json('dataset_es_dev.json', lines = True)
#dataset_dev.head()

In [3]:
#dataset_dev.shape

In [4]:
#dataset_dev.drop_duplicates(inplace = True) #no hay datos duplicados ni faltantes
#dataset_dev.shape

In [5]:
#dataset_dev.isna().sum()

In [6]:
#dataset_dev.language.value_counts() #solo reseñas en español

In [7]:
#dataset_dev.review_body.shape

In [8]:
#dataset_dev.stars.value_counts()

In [9]:
#dataset_dev.product_id.value_counts()

In [10]:
#dataset_dev.reviewer_id.value_counts() #product id y reviewer id no deberian coincidir?

Elijo una instancia al azar y veo el review boby

In [11]:
#np.random #.seed(56)
#index_random = np.random.randint(0,high = dataset_dev.shape[0])
#descripcion = dataset_dev.iloc[index_random].review_body
#print(index_random, descripcion)

vemos la estrella con la que califico al producto comprado

In [12]:
#print (f'El comprador {index_random}')
#print (f'Califico al producto con {dataset_dev.iloc[index_random].stars} estrellas' )

#print(index_random, dataset_dev.iloc[index_random].stars)

In [13]:
#nlp = spacy.load("es_core_news_sm")

In [14]:
#doc = nlp (descripcion)

In [15]:
#for token in doc:
#    print(token.text)

**Stopwords**

In [16]:
#from spacy.lang.es.stop_words import STOP_WORDS
#stopwords_spacy = list(STOP_WORDS)
#print (stopwords_spacy)
#len(stopwords_spacy)

Imprimimos las palabras del texto que no son stop words según Spacy

In [17]:
#for token in doc:
 #   if token.is_stop == False:
  #      print(token)

**Lemmatización**

In [18]:
#for token in doc:
 #   print (token.text, token.lemma_)

**POS Part of Speech**

In [19]:
#for token in doc:
#    print (token.text, token.pos_)

**Tokenización**

Eliminamos los signos 

In [20]:
#puntua = string.punctuation + '¿!¡? + " "'
#puntua

In [21]:
#def text_data_cleaning(sentence):
 #   doc=nlp(sentence)
    
 #   tokens = []
 #  for token in doc:
  #      if token.lemma_ != '-PRON-':
   #         temp= token.lemma_.strip()
    #    else:
#            temp = token
#        tokens.append(temp)
        
#    clean_tokens = []
#    for token in tokens:
#        if token not in stopwords_spacy and token not in puntua:
#            clean_tokens.append(token)
            
#    return " ".join(clean_tokens)

In [22]:
#text_data_cleaning(descripcion)

Aplicamos un text data cleaning sobre review body en todo el dataset y lo imprimimos

In [23]:
#dataset_dev.review_body.apply(text_data_cleaning)

In [24]:
#dataset_dev.head(3)

In [25]:
#dataset_dev['review_body']= dataset_dev.review_body.apply(text_data_cleaning)

In [26]:
#dataset_dev

Convertimos el Dataset a CSV para poder cargarlo mas rápido

In [27]:
#dataset_dev.to_csv('dataset_dev.csv')

In [28]:
#dataset_dev

In [29]:
dataset_dev = pd.read_csv('dataset_dev.csv') 
dataset_dev.head()

,Unnamed: 0,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,0,es_0417480,product_es_0873923,reviewer_es_0672978,1,malisimo grande aparatoso y protector pantalla,Mala compra,es,wireless
1,1,es_0180432,product_es_0713146,reviewer_es_0100858,1,recibir pedido devolución,No lo he recibido,es,apparel
2,2,es_0144850,product_es_0356874,reviewer_es_0486447,1,buscar sistema funcionar abrazadera agarrar ma...,Que no aprieta bien en el manillar,es,sports
3,3,es_0339629,product_es_0939832,reviewer_es_0894703,1,Utilicé broca menor diámetro y doblar mantequi...,Brocas de mantequilla,es,home_improvement
4,4,es_0858362,product_es_0489066,reviewer_es_0887663,1,gustar olor a viejo y aspecto malo,No me gusta,es,beauty


**Voy a transformar los datos de los dataset de TRAIN Y TEST y los voy a importar en CSV ya transformados**

In [30]:
dataset_train = pd.read_csv('dataset_train.csv') 
dataset_train.head()

,Unnamed: 0,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,0,es_0491108,product_es_0296024,reviewer_es_0999081,1,kar pantalla 8 mes y recibir respuesta fabricante,television Nevir,es,electronics
1,1,es_0869872,product_es_0922286,reviewer_es_0216771,1,Horrible comprar inglés informático hora capaz...,Dinero tirado a la basura con esta compra,es,electronics
2,2,es_0811721,product_es_0474543,reviewer_es_0929213,1,obligar a comprar unidad y llegar y forma recl...,solo llega una unidad cuando te obligan a comp...,es,drugstore
3,3,es_0359921,product_es_0656090,reviewer_es_0224702,1,entrar descalificar vendedor mes espera .... s...,PRODUCTO NO RECIBIDO.,es,wireless
4,4,es_0068940,product_es_0662544,reviewer_es_0224827,1,llegar y co talla equivocado,Devuelto,es,shoes


In [31]:
dataset_train

,Unnamed: 0,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,0,es_0491108,product_es_0296024,reviewer_es_0999081,1,kar pantalla 8 mes y recibir respuesta fabricante,television Nevir,es,electronics
1,1,es_0869872,product_es_0922286,reviewer_es_0216771,1,Horrible comprar inglés informático hora capaz...,Dinero tirado a la basura con esta compra,es,electronics
2,2,es_0811721,product_es_0474543,reviewer_es_0929213,1,obligar a comprar unidad y llegar y forma recl...,solo llega una unidad cuando te obligan a comp...,es,drugstore
3,3,es_0359921,product_es_0656090,reviewer_es_0224702,1,entrar descalificar vendedor mes espera .... s...,PRODUCTO NO RECIBIDO.,es,wireless
4,4,es_0068940,product_es_0662544,reviewer_es_0224827,1,llegar y co talla equivocado,Devuelto,es,shoes
...,...,...,...,...,...,...,...,...,...
199995,199995,es_0715276,product_es_0317036,reviewer_es_0643604,5,Mando funcionar perfectamente y cumplir funció...,Tal y como se describe,es,electronics
199996,199996,es_0085190,product_es_0622919,reviewer_es_0466173,5,Compré batería reticencia resultar fácil insta...,Funciona perfectamente,es,electronics
199997,199997,es_0484496,product_es_0358101,reviewer_es_0330744,5,calidad satisfecho compra,Buena calidad.,es,apparel
199998,199998,es_0930141,product_es_0788855,reviewer_es_0694290,5,Perfecto cumple hijo,Recomendado,es,toy


In [32]:
dataset_test = pd.read_csv('dataset_test.csv') 
dataset_test.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0038754,product_es_0113523,reviewer_es_0580071,1,llegar articulo mandar correo normal seguimien...,no me llego,es,wireless
1,es_0748979,product_es_0017036,reviewer_es_0819733,1,mensajería horrible compro,amazon sigue sin cumplir en las entregas,es,home
2,es_0411746,product_es_0138642,reviewer_es_0508607,1,decepcionado vendedor pedido llegar a y cumpli...,ESTAFA EN EL ENVÍO,es,toy
3,es_0786686,product_es_0170887,reviewer_es_0491157,1,valoración producto AMAZON ofreceis producto a...,Estafa de Amazon,es,home
4,es_0429700,product_es_0710642,reviewer_es_0008745,1,interés libro y probar versión kindle abrir po...,No conseguí pasar de la portada en Kindle,es,digital_ebook_purchase


In [33]:
dataset_test.shape

(5000, 8)

Tomamos una muestra de dataset_train para poder analizar mejor el DS por su cantidad de datos.

In [34]:
dataset_train_dos = dataset_train.sample( n= 1000, weights=np.ones(dataset_train.shape[0]), random_state = 42)
dataset_train_dos

,Unnamed: 0,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
74908,74908,es_0101647,product_es_0450032,reviewer_es_0608549,2,embalar retrasar,Viene sucio,es,home
190142,190142,es_0302856,product_es_0907235,reviewer_es_0714115,5,regalamos a sobrina 12 año fan serie encantar ...,Para un regalo,es,jewelry
146398,146398,es_0948829,product_es_0063764,reviewer_es_0178095,4,correa tardar llegar a merecer pena,Está muy bien,es,sports
119731,119731,es_0065556,product_es_0845916,reviewer_es_0636981,3,probar quitar punto negro probé farmacia y noche,No es muy bueno,es,beauty
31203,31203,es_0009633,product_es_0155896,reviewer_es_0229708,1,quedar satisfecho modelo 2019 y especifique fu...,Funda y movil incompatibles,es,home
...,...,...,...,...,...,...,...,...,...
27363,27363,es_0078673,product_es_0270175,reviewer_es_0358567,1,poner él evitar presencia mota y fácil instala...,Nada recomendable,es,wireless
190047,190047,es_0144389,product_es_0785813,reviewer_es_0369290,5,Perfecta niño y adolescente ocupar espacio y c...,Perfecta,es,furniture
89201,89201,es_0805021,product_es_0167570,reviewer_es_0114986,3,irritar piel crema,A mi piel no le funciona,es,beauty
37053,37053,es_0680350,product_es_0790523,reviewer_es_0367273,1,compro y venir rueda eje querer devolver él pe...,Espero respuesta de Amazon al comentario.,es,lawn_and_garden


In [35]:
dataset_train.stars.value_counts()

1    40000
2    40000
3    40000
4    40000
5    40000
Name: stars, dtype: int64

In [36]:
dataset_train_dos.stars.value_counts()

1    225
5    199
2    196
3    193
4    187
Name: stars, dtype: int64

**Vectorización TF-IDF**

Tomamos X del DS train y del DS dev y comparamos su desempeño

In [39]:
tfidf = TfidfVectorizer (analyzer = 'word', max_features = 5000)
X_ds_train = tfidf.fit_transform(dataset_train_dos['review_body'].values.astype('U')) 
y_ds_train = dataset_train_dos ['stars']

In [40]:
X_ds_dev = tfidf.transform(dataset_dev['review_body'].values.astype('U')) 
y_ds_dev = dataset_dev['stars']

In [41]:
y_ds_train

74908     2
190142    5
146398    4
119731    3
31203     1
         ..
27363     1
190047    5
89201     3
37053     1
108388    3
Name: stars, Length: 1000, dtype: int64

**OVO - One vs One Classifier**

In [43]:
modelo_train= OneVsOneClassifier(LinearSVC(random_state = 42)).fit(X_ds_train,y_ds_train)
y_dev_pred= modelo_train.predict(X_ds_dev)

In [44]:
modelo_train

OneVsOneClassifier(estimator=LinearSVC(random_state=42))

In [45]:
y_dev_pred

array([2, 1, 4, ..., 4, 4, 4], dtype=int64)

In [46]:
print(classification_report(y_ds_dev, y_dev_pred))

              precision    recall  f1-score   support

           1       0.43      0.57      0.49      1000
           2       0.26      0.27      0.27      1000
           3       0.28      0.26      0.27      1000
           4       0.31      0.25      0.28      1000
           5       0.45      0.40      0.42      1000

    accuracy                           0.35      5000
   macro avg       0.35      0.35      0.35      5000
weighted avg       0.35      0.35      0.35      5000

